In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# !cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
!pip install optuna
!pip install catboost
!pip install skranger
!pip install ngboost
!pip install lightgbm
!pip install hyperopt

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import random
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tqdm

from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

from lightgbm import LGBMRegressor
from ngboost import NGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge
from catboost import CatBoostRegressor, Pool
from skranger.ensemble import RangerForestRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from ngboost.scores import LogScore

from hyperopt import fmin, hp, tpe
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

In [3]:
class Config:
  seed = 42
  epochs = 200

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [5]:
def dataset_split_X_y(df):    
    """
    @Description: split data into features and labels
    @Param: df, pandas dataframe with columns starting with X for features and Y for labels
    @Return: features and labels in pandas dataframes
    """
    xs = df.filter(regex='X') # Input : X Feature
    ys = df.filter(regex='Y') # Output : Y Feature
    return xs, ys

In [6]:
def check_for_NAs(df, show=False):
    """
    @Description: checks for the NAs in the dataframe
    @Param1: df, pandas dataframe
    @Param2: show, boolean indicating whether NaN data are also necessary as a part of the output
    @Return: name of the columns with NaN
    """
    nan_values = df.loc[:, df.isnull().any()]
    if show:
        return df[df.isna().any(axis=1)]
    return list(nan_values.columns)

In [7]:
def check_for_label_bound(df, labels, bound):
    """
    @Description: check bound is inbetween min and max
    @Param1: df, pandas dataframe
    @Param2: labels, list of column names 
    @Param3: thres: list of bounds
    @Return: names of the columns not within the bound
    """
    n = len(labels)
    result = []
    for idx in range(n):
        col = labels[idx]
        thres = bound[idx]
        extracted_column = df[col]
        if not extracted_column.between(thres[0], thres[1]).all():
            result.append(labels[idx])
    if len(result) == 0:
        print('everything is within the bound')
    return result

In [8]:
def zero_variance(df):
    """
    @Description: check for zero_variance
    @Param1: df, pandas dataframe
    @Return: names of the columns with zero variance
    """
    result = []
    for col in df.columns:
        if df[col].var() == 0:
            result.append(col)
    return result

In [9]:
def get_top_correlation(df, n=10):
    """
    @Description: print out top correlated features
    @Param1: df, pandas dataframe
    @Param2: n, number of lines to print 
    @Return: pandas series
    """
    pairs = set()
    for idx1 in range(0, df.shape[1]):
        for idx2 in range(0, idx1+1):
            pairs.add((df.columns[idx1], df.columns[idx2]))
    corr = df.corr().abs().unstack()
    corr = corr.drop(labels=pairs).sort_values(ascending=False)
    return corr[0:n]

In [10]:
def adjacent_histogram_boxplot(feature_var, figsize = (7, 5)):
    """
    @Description: plot histogram and boxplot in next to each other
    @Param1: feature_var, pandas series 
    @Param2: figsize, size of the figure 
    """
    fig, (hist_plot, box_plot) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios':(.85,.15)}, figsize=figsize)
    sns.distplot(feature_var, kde=True, ax=hist_plot, kde_kws= {"linewidth":1.5}) 
    sns.boxplot(feature_var, ax=box_plot, linewidth = 1, width = 0.5)
    hist_plot.set_ylabel('')    
    hist_plot.set_xlabel('')
    box_plot.set_xlabel('')
    hist_plot.tick_params(labelsize=8)
    box_plot.tick_params(labelsize=8)
    fig.suptitle(feature_var.name, fontsize = 10)
    hist_plot.axvline(np.mean(feature_var),color='red',linestyle='-',lw = 1.5)
    hist_plot.axvline(np.median(feature_var),color='green',linestyle='--',lw = 1.5)
    

In [11]:
def lg_nrmse(gt, preds):
    """
    @Description: Metric used in this project
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    preds = pd.DataFrame(preds)
    all_nrmse = []
    for idx in range(0,14):
        rmse = mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [12]:
def lg_individual_nrmse(gt, preds):
    """
    @Description: Metric used in this project (individual)
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    rmse = mean_squared_error(gt, preds, squared=False)
    nrmse = rmse/np.mean(np.abs(gt))
    return nrmse

In [13]:
def find_outlier_zscore(data, threshold = 3):
    mean = np.mean(data)
    std = np.std(data)
    zs = [(y - mean) / std for y in data]
    masks = np.where(np.abs(zs) > threshold)
    return masks[0]

In [14]:
ys = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 
      'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10', 
      'Y_11', 'Y_12', 'Y_13', 'Y_14']
ys_bounds = [[0.2, 2], [0.2, 2.1], [0.2, 2.1], 
             [7, 19], [22, 36.5], [-19.2, 19], 
             [2.4, 4], [-29.2, -24], [-29.2, -24],
             [-30.6, -20], [19.6, 26.6], [-29.2, -24],
             [-29.2, -24], [-29.2, -24]]

In [15]:
seed_everything(Config.seed)

train_df = pd.read_csv('C:/Users/computer/Desktop/새 폴더./train.csv')
test_x = pd.read_csv('C:/Users/computer/Desktop/새 폴더./test.csv')
train_x, train_y = dataset_split_X_y(train_df)

cols_with_zero_variance = zero_variance(train_x) # 분산이 0 (통과 여부)
train_x = train_x.drop(cols_with_zero_variance, axis = 1)
test_x = test_x.drop(cols_with_zero_variance, axis = 1)

train_x = train_x.drop(['X_10', 'X_11'], axis = 1) # 결측치가 많음 (결측치 = 0, 공지사항)
test_x = test_x.drop(['X_10', 'X_11'], axis = 1)

test_x = test_x.drop('ID', axis=1)

In [16]:
class Config:
  seed = 42
  epochs = 200
  cv=10
  test_size = 0.2

#stacking

In [18]:
 def get_stacking_base_datasets(model, train_x, train_y, col,test, params):
     kf = KFold(n_splits=Config.cv, shuffle=False)
     train_fold_pred = np.zeros((train_x.shape[0],1))
     test_pred = np.zeros((test.shape[0],Config.cv))
    
    
     for folder_counter, (train_index, valid_index) in enumerate(kf.split(train_x)):
         print('Fold : ', folder_counter, ' Start')
         X_tr = train_x.loc[train_index]
         y_tr = train_y[col].loc[train_index]
         X_te = train_x.loc[valid_index] 
       
         if model == 'cat':
           model = CatBoostRegressor(random_state=1,
                                     **params)
       
         elif model == 'extra':
           model = ExtraTreesRegressor(random_state=1, 
                                       **params)

         elif model == 'ngbr':
           model = NGBRegressor(random_state = 1)
       
         elif model == 'lgbm':
           model = LGBMRegressor(random_state=1, n_jobs=-1, 
                                 **params)

         model.fit(X_tr, y_tr)
         train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1) 
         test_pred[:, folder_counter] = model.predict(test) 
        
     test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
     return train_fold_pred, test_pred_mean 

In [ ]:
# Y_01
#min_samples_split

cat_01 = {'depth': 9, 'fold_len_multiplier': 1.6722688563924544, 'l2_leaf_reg': 9.992348977307927, 'learning_rate': 0.03686783566671033, 'max_bin': 16, 'min_data_in_leaf': 7, 'n_estimators': 500, 'random_strength': 0.5478002316160607}

extra_01 = {'bootstrap': 0, 'ccp_alpha': 0.5956203348598316, 'max_depth': 43, 'max_features': 1, 'max_leaf_nodes': 22, 'min_impurity_decrease': 195.59697998782488, 'min_samples_leaf': 0.4, 'min_samples_split': 0.427, 'min_weight_fraction_leaf': 0.3872874854064327, 'n_estimators': 200}

lgbm_01 = {'colsample_bytree': 0.572280100273023, 'learning_rate': 0.010283635038627429, 'max_depth': 180, 'min_child_samples': 135, 'min_split_gain': 0.04511227284338413, 'n_estimators': 900, 'num_leaves': 70, 'reg_alpha': 4.406681827912319, 'reg_lambda': 20.4785600448913, 'scale_pos_weight': 8.302374117433086, 'subsample': 0.1688669888026464}

ngbr_01 = {'n_estimators': 250, 'learning_rate': 0.027115337704182965, 'natural_gradient': True, 'col_sample': 0.2, 'minibatch_frac': 0.8, 'tol': 5.5136412071576055e-05}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_01', test=test_x, params = cat_01)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_01', test=test_x, params = extra_01)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_01', test=test_x, params = lgbm_01)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_01', test=test_x, params = ngbr_01)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_01)
lr_final.fit(Stack_final_X_train, train_y['Y_01'])
stack_final1 = lr_final.predict(Stack_final_X_test)




# Y_02
#min_samples_split

cat_02 = {'depth': 9, 'fold_len_multiplier': 1.8379420467251593, 'l2_leaf_reg': 27.102344731579784, 'learning_rate': 0.03597820559455176, 'max_bin': 14, 'min_data_in_leaf': 2, 'n_estimators': 500, 'random_strength': 0.9800368067026318}

extra_02 = {'bootstrap': 0, 'ccp_alpha': 0.9020983921597531, 'max_depth': 42, 'max_features': 2, 'max_leaf_nodes': 7, 'min_impurity_decrease': 150.7925115966371, 'min_samples_leaf': 18, 'min_samples_split': 15, 'min_weight_fraction_leaf': 0.3321219768527379, 'n_estimators': 200}

lgbm_02 =  {'colsample_bytree': 0.7641322280477741, 'learning_rate': 0.010977205425053654, 'max_depth': 90, 'min_child_samples': 75, 'min_split_gain': 0.13379952895779884, 'n_estimators': 900, 'num_leaves': 80, 'reg_alpha': 1.9214119194170154, 'reg_lambda': 14.454450236504218, 'scale_pos_weight': 2.171961031806387, 'subsample': 0.9552593593877317}

ngbr_02 = {'n_estimators': 250, 'learning_rate': 0.0656349966273891, 'natural_gradient': True, 'col_sample': 0.8, 'minibatch_frac': 0.55, 'tol': 0.00028029350235701545}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_02', test=test_x, params = cat_02)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_02', test=test_x, params = extra_02)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_02', test=test_x, params = lgbm_02)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_02', test=test_x, params = ngbr_02)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_02)
lr_final.fit(Stack_final_X_train, train_y['Y_02'])
stack_final2 = lr_final.predict(Stack_final_X_test)




# Y_03
#min_samples_split

cat_03 = {'depth': 7, 'fold_len_multiplier': 2.498263900973405, 'l2_leaf_reg': 36.312606304859514, 'learning_rate': 0.043147429358500425, 'max_bin': 14, 'min_data_in_leaf': 2, 'n_estimators': 450, 'random_strength': 0.6650471273750369}

extra_03 = {'bootstrap': 1, 'ccp_alpha': 0.9068334703113171, 'max_depth': 22, 'max_features': 2, 'max_leaf_nodes': 11, 'min_impurity_decrease': 6.407983868655598, 'min_samples_leaf': 17, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.4864839919729328, 'n_estimators': 1450}

lgbm_03 = {'colsample_bytree': 0.5504769098255781,  'learning_rate': 0.019653385015120244, 'max_depth': 220, 'min_child_samples': 25, 'min_split_gain': 0.1273611040963466, 'n_estimators': 470, 'num_leaves': 160, 'reg_alpha': 3.5549669150756706, 'reg_lambda': 39.88636182674132, 'scale_pos_weight': 12.46696320152359, 'subsample': 0.7590007450921917}

ngbr_03 = {'n_estimators': 410, 'learning_rate': 0.11407060690033853, 'natural_gradient': True, 'col_sample': 0.25, 'minibatch_frac': 1, 'tol': 0.000166350313681024}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_03', test=test_x, params = cat_03)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_03', test=test_x, params = extra_03)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_03', test=test_x, params = lgbm_03)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_03', test=test_x, params = ngbr_03)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_03)
lr_final.fit(Stack_final_X_train, train_y['Y_03'])
stack_final3 = lr_final.predict(Stack_final_X_test)



# Y_04
#min_samples_split

cat_04 = {'depth': 10, 'fold_len_multiplier': 1.3980250309157025, 'l2_leaf_reg': 5.838850864558845, 'learning_rate': 0.2737350830778467, 'max_bin': 78, 'min_data_in_leaf': 561, 'n_estimators': 20, 'random_strength': 0.009672204431612739}

extra_04 = {'bootstrap': 0, 'ccp_alpha': 0.7076108565007323, 'max_depth': 48, 'max_features': 3, 'max_leaf_nodes': 16, 'min_impurity_decrease': 45.084212905659186, 'min_samples_leaf': 11, 'min_samples_split': 20, 'min_weight_fraction_leaf': 0.1164511175367393, 'n_estimators': 100}

lgbm_04 = {'colsample_bytree': 0.5597537952569402, 'learning_rate': 0.02374663979814546, 'max_depth': 32, 'min_child_samples': 100, 'min_split_gain': 0.12211426885216736, 'n_estimators': 1263, 'num_leaves': 200, 'reg_alpha': 14.606693962963451, 'reg_lambda': 299.52278825209424, 'scale_pos_weight': 7.7785016838070735, 'subsample': 0.6254745287838821}

ngbr_04 = {'n_estimators': 431, 'learning_rate': 0.08883519586581468, 'natural_gradient': True, 'col_sample': 0.97, 'minibatch_frac': 0.75, 'tol': 8.896682623929568e-05}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_04', test=test_x, params = cat_04)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_04', test=test_x, params = extra_04)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_04', test=test_x, params = lgbm_04)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_04', test=test_x, params = ngbr_04)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_04)
lr_final.fit(Stack_final_X_train, train_y['Y_04'])
stack_final4 = lr_final.predict(Stack_final_X_test)



# Y_05
#min_samples_split

cat_05 = {'depth': 9, 'fold_len_multiplier': 1.7527609013156893, 'l2_leaf_reg': 5.150371128645829, 'learning_rate': 0.23166991521375363, 'max_bin': 181, 'min_data_in_leaf': 591, 'n_estimators': 17, 'random_strength': 0.08626442162325075}

extra_05 = {'bootstrap': 0, 'ccp_alpha': 0.17223432236304015, 'max_depth': 39, 'max_features': 2, 'max_leaf_nodes': 16, 'min_impurity_decrease': 166.70077338146032, 'min_samples_leaf': 34, 'min_samples_split': 15, 'min_weight_fraction_leaf': 0.22146453407955657, 'n_estimators': 200}

lgbm_05 = {'colsample_bytree': 0.4311015575880258, 'learning_rate': 0.01749725932551278, 'max_depth': 53, 'min_child_samples': 15, 'min_split_gain': 0.2820951740673634, 'n_estimators': 974, 'num_leaves': 165, 'reg_alpha': 9.604623064885754, 'reg_lambda': 12.314490508636432, 'scale_pos_weight': 6.6422956907936825, 'subsample': 0.7390190399971659}

ngbr_05 = {'n_estimators': 449, 'learning_rate': 0.08850472848590257, 'natural_gradient': True, 'col_sample': 0.8, 'minibatch_frac': 0.97, 'tol': 7.048508838263751e-05}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_05', test=test_x, params = cat_05)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_05', test=test_x, params = extra_05)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_05', test=test_x, params = lgbm_05)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_05', test=test_x, params = ngbr_05)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_05)
lr_final.fit(Stack_final_X_train, train_y['Y_05'])
stack_final5 = lr_final.predict(Stack_final_X_test)



# Y_06
#min_samples_split

cat_06 = {'depth': 8, 'fold_len_multiplier': 1.4734463589684192, 'l2_leaf_reg': 7.343561976614034, 'learning_rate': 0.2546701358021111, 'max_bin': 8, 'min_data_in_leaf': 280, 'n_estimators': 13, 'random_strength': 0.06423202371274109}

extra_06 = {'bootstrap': 0, 'ccp_alpha': 0.263122350467869, 'max_depth': 12, 'max_features': 1, 'max_leaf_nodes': 6, 'min_impurity_decrease': 140.20905071348278, 'min_samples_leaf': 50, 'min_samples_split': 50, 'min_weight_fraction_leaf': 0.17986464144348094, 'n_estimators': 450}

lgbm_06 = {'colsample_bytree': 0.6889745043181079, 'learning_rate': 0.06146161938790444, 'max_depth': 89, 'min_child_samples': 10, 'min_split_gain': 0.669592868575692, 'n_estimators': 1169, 'num_leaves': 175, 'reg_alpha': 11.405277636150856, 'reg_lambda': 112.37954230084294, 'scale_pos_weight': 5.932435783263877, 'subsample': 0.8265223228903998}

ngbr_06 = {'n_estimators': 153, 'learning_rate': 0.05121743231435252, 'natural_gradient': True, 'col_sample': 0.7000000000000001, 'minibatch_frac': 0.97, 'tol': 0.00013297533787653073}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_06', test=test_x, params = cat_06)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_06', test=test_x, params = extra_06)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_06', test=test_x, params = lgbm_06)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_06', test=test_x, params = ngbr_06)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_06)
lr_final.fit(Stack_final_X_train, train_y['Y_06'])
stack_final6 = lr_final.predict(Stack_final_X_test)



# Y_07
#min_samples_split

cat_07 = {'n_estimators': 350, 'depth': 9, 'learning_rate': 0.07700980062937977, 'l2_leaf_reg': 7.366594895982364, 'max_bin': 80, 'min_data_in_leaf': 654, 'random_strength': 0.12106590929604114, 'fold_len_multiplier': 2.3574644740356874}

extra_07 = {'n_estimators': 100, 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 7, 'min_weight_fraction_leaf': 0.15163758513069608, 'max_features': 'auto', 'max_leaf_nodes': 16, 'min_impurity_decrease': 28.474990166012663, 'bootstrap': True, 'ccp_alpha': 0.8351211481986806}

lgbm_07 = {'colsample_bytree': 0.8663251864650988, 'learning_rate': 0.018110306887688978, 'max_depth': 166, 'min_child_samples': 50, 'min_split_gain': 0.025403061552667243, 'n_estimators': 1080, 'num_leaves': 100, 'reg_alpha': 2.0131018839563666, 'reg_lambda': 63.56640846106552, 'scale_pos_weight': 1.8584564419776715, 'subsample': 0.7643028435523616}

ngbr_07 = {'n_estimators': 280, 'learning_rate': 0.0543058099317307, 'natural_gradient': False, 'col_sample': 0.61, 'minibatch_frac': 0.53, 'tol': 4.259736507913848e-06}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_07', test=test_x, params = cat_07)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_07', test=test_x, params = extra_07)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_07', test=test_x, params = lgbm_07)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_07', test=test_x, params = ngbr_07)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_07)
lr_final.fit(Stack_final_X_train, train_y['Y_07'])
stack_final7 = lr_final.predict(Stack_final_X_test)



# Y_08
#min_samples_split

cat_08 = {'n_estimators': 450, 'depth': 11, 'learning_rate': 0.06098987016322603, 'l2_leaf_reg': 7.78479685048665, 'max_bin': 244, 'min_data_in_leaf': 423, 'random_strength': 0.4646435148235979, 'fold_len_multiplier': 1.7857138740606202}

extra_08 = {'n_estimators': 150, 'max_depth': 31, 'min_samples_split': 45, 'min_samples_leaf': 38, 'min_weight_fraction_leaf': 0.3764820550107259, 'max_features': 'log2', 'max_leaf_nodes': 21, 'min_impurity_decrease': 41.54716726233874, 'bootstrap': True, 'ccp_alpha': 0.9985147070627858}

lgbm_08 = {'colsample_bytree': 0.8970390757241629, 'learning_rate': 0.03571726260659087, 'max_depth': 164, 'min_child_samples': 30, 'min_split_gain': 0.2863362850926679, 'n_estimators': 740, 'num_leaves': 100, 'reg_alpha': 1.1167159754886287, 'reg_lambda': 280.9798636389436, 'scale_pos_weight': 4.75867892931176, 'subsample': 0.681716202670263}

ngbr_08 = {'n_estimators': 490, 'learning_rate': 0.04770929499260395, 'natural_gradient': True, 'col_sample': 1, 'minibatch_frac': 0.6, 'tol': 0.00019175990157775972}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_08', test=test_x, params = cat_08)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_08', test=test_x, params = extra_08)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_08', test=test_x, params = lgbm_08)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_08', test=test_x, params = ngbr_08)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_08)
lr_final.fit(Stack_final_X_train, train_y['Y_08'])
stack_final8 = lr_final.predict(Stack_final_X_test)



# Y_09
#min_samples_split

cat_09 = {'n_estimators': 450, 'depth': 7, 'learning_rate': 0.08417441728687995, 'l2_leaf_reg': 7.003716950710109, 'max_bin': 249, 'min_data_in_leaf': 85, 'random_strength': 0.01346018468768982, 'fold_len_multiplier': 1.8510933222341048}

extra_09 = {'n_estimators': 50, 'max_depth': 35, 'min_samples_split': 45, 'min_samples_leaf': 11, 'min_weight_fraction_leaf': 0.1446169507139342, 'max_features': None, 'max_leaf_nodes': 10, 'min_impurity_decrease': 190.68425357881898, 'bootstrap': True, 'ccp_alpha': 0.8503455224547213}

lgbm_09 = {'n_estimators': 900, 'max_depth': 86, 'num_leaves': 150, 'min_child_samples': 85, 'colsample_bytree': '0.90507', 'subsample': '0.62362', 'min_split_gain': '0.21034', 'scale_pos_weight': '8.77311', 'reg_alpha': '0.07069', 'reg_lambda': '499.10672', 'learning_rate': '0.04679'}

ngbr_09 = {'n_estimators': 250, 'learning_rate': 0.0531679007741611, 'natural_gradient': False, 'col_sample': 0.8200000000000001, 'minibatch_frac': 0.65, 'tol': 0.00013547208288125422}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_09', test=test_x, params = cat_09)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_09', test=test_x, params = extra_09)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_09', test=test_x, params = lgbm_09)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_09', test=test_x, params = ngbr_09)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_09)
lr_final.fit(Stack_final_X_train, train_y['Y_09'])
stack_final9 = lr_final.predict(Stack_final_X_test)



# Y_10
#min_samples_split

cat_10 = {'depth': 10, 'fold_len_multiplier': 1.8964686014236263, 'l2_leaf_reg': 6.890506058346803, 'learning_rate': 0.04431783808011194, 'max_bin': 135, 'min_data_in_leaf': 449, 'n_estimators': 400, 'random_strength': 0.0715293516760773}

extra_10 = {'bootstrap': 0, 'ccp_alpha': 0.06816154953537701, 'max_depth': 34, 'max_features': 2, 'max_leaf_nodes': 17, 'min_impurity_decrease': 10.897433724670414, 'min_samples_leaf': 45, 'min_samples_split': 25, 'min_weight_fraction_leaf': 0.22786627091058692, 'n_estimators': 100}

lgbm_10 = {'colsample_bytree': 0.8350973419202665, 'learning_rate': 0.03134966396365972, 'max_depth': 114, 'min_child_samples': 20, 'min_split_gain': 0.24406788869557822, 'n_estimators': 454, 'num_leaves': 115, 'reg_alpha': 1.0870546166564243, 'reg_lambda': 346.21163772786895, 'scale_pos_weight': 5.81617865285278, 'subsample': 0.45612075761336973}

ngbr_10 = {'n_estimators': 400, 'learning_rate': 0.10343505554950799, 'natural_gradient': True, 'col_sample': 0.71, 'minibatch_frac': 0.96, 'tol': 6.075001314642106e-05}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_10', test=test_x, params = cat_10)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_10', test=test_x, params = extra_10)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_10', test=test_x, params = lgbm_10)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_10', test=test_x, params = ngbr_10)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_10)
lr_final.fit(Stack_final_X_train, train_y['Y_10'])
stack_final10 = lr_final.predict(Stack_final_X_test)



# Y_11
#min_samples_split

cat_11 = {'depth': 9, 'fold_len_multiplier': 1.469922031736939, 'l2_leaf_reg': 5.365254126430433, 'learning_rate': 0.05182013518976086, 'max_bin': 147, 'min_data_in_leaf': 238, 'n_estimators': 450, 'random_strength': 0.07839813420603847}

extra_11 = {'bootstrap': 0, 'ccp_alpha': 0.22748332620407474, 'max_depth': 18, 'max_features': 2, 'max_leaf_nodes': 30, 'min_impurity_decrease': 63.03156125087142, 'min_samples_leaf': 6, 'min_samples_split': 20, 'min_weight_fraction_leaf': 0.04105676583350759, 'n_estimators': 150}

lgbm_11 = {'colsample_bytree': 0.7285829045071064, 'learning_rate': 0.019839273085108612, 'max_depth': 71, 'min_child_samples': 50, 'min_split_gain': 0.35567737788276876, 'n_estimators': 970, 'num_leaves': 140, 'reg_alpha': 0.27353134227182774, 'reg_lambda': 157.85749037224548, 'scale_pos_weight': 5.956126991298146, 'subsample': 0.7509931500532172}

ngbr_11 = {'n_estimators': 330, 'learning_rate': 0.10650808882845716, 'natural_gradient': False, 'col_sample': 0.67, 'minibatch_frac': 0.8200000000000001, 'tol': 5.808799526610105e-05}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_11', test=test_x, params = cat_11)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_11', test=test_x, params = extra_11)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_11', test=test_x, params = lgbm_11)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_11', test=test_x, params = ngbr_11)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_11)
lr_final.fit(Stack_final_X_train, train_y['Y_11'])
stack_final11 = lr_final.predict(Stack_final_X_test)



# Y_12
#min_samples_split

cat_12 = {'n_estimators': 250, 'depth': 6, 'learning_rate': 0.15612168413836122, 'l2_leaf_reg': 4.521702132180398, 'max_bin': 249, 'min_data_in_leaf': 218, 'random_strength': 4.051962608010968, 'fold_len_multiplier': 1.1690440537893567}

extra_12 = {'bootstrap': 0, 'ccp_alpha': 0.7514591922993081, 'max_depth': 50, 'max_features': 3, 'max_leaf_nodes': 3, 'min_impurity_decrease': 112.61937027705285, 'min_samples_leaf': 25, 'min_samples_split': 30, 'min_weight_fraction_leaf': 0.16488737437051704, 'n_estimators': 100}

lgbm_12 = {'colsample_bytree': 0.6115826698158419, 'learning_rate': 0.010052927231718068, 'max_depth': 71, 'min_child_samples': 85, 'min_split_gain': 0.12003011548878659, 'n_estimators': 1300, 'num_leaves': 120, 'reg_alpha': 1.3013867029804251, 'reg_lambda': 269.3915696845848, 'scale_pos_weight': 5.290961082236748, 'subsample': 0.7542724715058367}

ngbr_12 = {'col_sample': 0.79, 'learning_rate': 0.11842716149209648, 'minibatch_frac': 0.78, 'n_estimators': 100, 'natural_gradient': 1, 'tol': 0.00017488848399995865}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_12', test=test_x, params = cat_12)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_12', test=test_x, params = extra_12)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_12', test=test_x, params = lgbm_12)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_12', test=test_x, params = ngbr_12)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_12)
lr_final.fit(Stack_final_X_train, train_y['Y_12'])
stack_final12 = lr_final.predict(Stack_final_X_test)



# Y_13
#min_samples_split

cat_13 = {'n_estimators': 250, 'depth': 10, 'learning_rate': 0.06019223910388208, 'l2_leaf_reg': 3.692745142783531, 'max_bin': 30, 'min_data_in_leaf': 71, 'random_strength': 0.061877335687993515, 'fold_len_multiplier': 2.1969765304562054}

extra_13 = {'bootstrap': 0, 'ccp_alpha': 0.9305876780539839, 'max_depth': 32, 'max_features': 3, 'max_leaf_nodes': 6, 'min_impurity_decrease': 103.59182427685685, 'min_samples_leaf': 41, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.2746586014429824, 'n_estimators': 100}

lgbm_13 = {'colsample_bytree': 0.9511047907962863, 'learning_rate': 0.023257873709858216, 'max_depth': 58, 'min_child_samples': 80, 'min_split_gain': 0.21488153574891886, 'n_estimators': 1300, 'num_leaves': 150, 'reg_alpha': 0.33761852089148814, 'reg_lambda': 57.05291849099506, 'scale_pos_weight': 2.0801436555772854, 'subsample': 0.5580106548214563}

ngbr_13 = {'n_estimators': 100, 'learning_rate': 0.09415734373605988, 'natural_gradient': False, 'col_sample': 0.67, 'minibatch_frac': 0.86, 'tol': 0.00010625760957734133}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_13', test=test_x, params = cat_13)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_13', test=test_x, params = extra_13)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_13', test=test_x, params = lgbm_13)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_13', test=test_x, params = ngbr_13)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_13)
lr_final.fit(Stack_final_X_train, train_y['Y_13'])
stack_final13 = lr_final.predict(Stack_final_X_test)



# Y_14
#min_samples_split

cat_14 = {'n_estimators': 250, 'depth': 9, 'learning_rate': 0.0994089124916012, 'l2_leaf_reg': 7.430077445061214, 'max_bin': 163, 'min_data_in_leaf': 420, 'random_strength': 0.5517727972303491, 'fold_len_multiplier': 1.1173814975860605}

extra_14 = {'bootstrap': 0, 'ccp_alpha': 0.5803551586810121, 'max_depth': 9, 'max_features': 0, 'max_leaf_nodes': 24, 'min_impurity_decrease': 114.7107697231918, 'min_samples_leaf': 6, 'min_samples_split': 35, 'min_weight_fraction_leaf': 0.1492044083631023, 'n_estimators': 150}

lgbm_14 = {'colsample_bytree': 0.8851122740930837, 'learning_rate': 0.013136814152245062, 'max_depth': 249, 'min_child_samples': 65, 'min_split_gain': 0.2072264172906347, 'n_estimators': 450, 'num_leaves': 135, 'reg_alpha': 0.642890771203696, 'reg_lambda': 45.624663648443345, 'scale_pos_weight': 6.400746088779947, 'subsample': 0.30084274480143686}

ngbr_14 = {'n_estimators': 300, 'learning_rate': 0.08509952436476127, 'natural_gradient': False, 'col_sample': 1, 'minibatch_frac': 0.86, 'tol': 0.00011282289882632527}

xx_train, xx_test = get_stacking_base_datasets('cat', train_x, train_y, col='Y_14', test=test_x, params = cat_14)
yy_train, yy_test = get_stacking_base_datasets('extra', train_x, train_y, col='Y_14', test=test_x, params = extra_14)
zz_train, zz_test = get_stacking_base_datasets('lgbm', train_x, train_y, col='Y_14', test=test_x, params = lgbm_14)
qq_train, qq_test = get_stacking_base_datasets('ngbr', train_x, train_y, col='Y_14', test=test_x, params = ngbr_14)


Stack_final_X_train = np.concatenate((xx_train, yy_train, zz_train, qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test, yy_test, zz_test, qq_test), axis=1)

# final_model 선택해야함
lr_final = LGBMRegressor(**lgbm_14)
lr_final.fit(Stack_final_X_train, train_y['Y_14'])
stack_final14 = lr_final.predict(Stack_final_X_test)

Fold :  0  Start
0:	learn: 0.3541160	total: 177ms	remaining: 1m 28s
1:	learn: 0.3535964	total: 184ms	remaining: 45.8s
2:	learn: 0.3530669	total: 190ms	remaining: 31.6s
3:	learn: 0.3525688	total: 197ms	remaining: 24.4s
4:	learn: 0.3521315	total: 203ms	remaining: 20.1s
5:	learn: 0.3516831	total: 209ms	remaining: 17.2s
6:	learn: 0.3512362	total: 215ms	remaining: 15.2s
7:	learn: 0.3508084	total: 221ms	remaining: 13.6s
8:	learn: 0.3503946	total: 228ms	remaining: 12.4s
9:	learn: 0.3500228	total: 234ms	remaining: 11.5s
10:	learn: 0.3496508	total: 240ms	remaining: 10.7s
11:	learn: 0.3493062	total: 246ms	remaining: 10s
12:	learn: 0.3489633	total: 253ms	remaining: 9.46s
13:	learn: 0.3485986	total: 259ms	remaining: 8.99s
14:	learn: 0.3482401	total: 266ms	remaining: 8.59s
15:	learn: 0.3479290	total: 272ms	remaining: 8.22s
16:	learn: 0.3476589	total: 278ms	remaining: 7.88s
17:	learn: 0.3473644	total: 284ms	remaining: 7.6s
18:	learn: 0.3470604	total: 290ms	remaining: 7.35s
19:	learn: 0.3467764	total

170:	learn: 0.3267389	total: 1.28s	remaining: 2.46s
171:	learn: 0.3266787	total: 1.28s	remaining: 2.45s
172:	learn: 0.3265588	total: 1.29s	remaining: 2.44s
173:	learn: 0.3264509	total: 1.3s	remaining: 2.43s
174:	learn: 0.3263716	total: 1.3s	remaining: 2.42s
175:	learn: 0.3263140	total: 1.31s	remaining: 2.41s
176:	learn: 0.3262404	total: 1.32s	remaining: 2.4s
177:	learn: 0.3261552	total: 1.32s	remaining: 2.39s
178:	learn: 0.3261124	total: 1.33s	remaining: 2.38s
179:	learn: 0.3260193	total: 1.34s	remaining: 2.38s
180:	learn: 0.3259333	total: 1.34s	remaining: 2.37s
181:	learn: 0.3258612	total: 1.35s	remaining: 2.36s
182:	learn: 0.3257759	total: 1.35s	remaining: 2.35s
183:	learn: 0.3257273	total: 1.36s	remaining: 2.34s
184:	learn: 0.3256520	total: 1.37s	remaining: 2.33s
185:	learn: 0.3255274	total: 1.37s	remaining: 2.32s
186:	learn: 0.3254554	total: 1.38s	remaining: 2.31s
187:	learn: 0.3253734	total: 1.39s	remaining: 2.3s
188:	learn: 0.3253250	total: 1.39s	remaining: 2.29s
189:	learn: 0.32

352:	learn: 0.3130829	total: 2.41s	remaining: 1s
353:	learn: 0.3130219	total: 2.42s	remaining: 997ms
354:	learn: 0.3129422	total: 2.42s	remaining: 990ms
355:	learn: 0.3128643	total: 2.43s	remaining: 983ms
356:	learn: 0.3127696	total: 2.44s	remaining: 976ms
357:	learn: 0.3127031	total: 2.44s	remaining: 969ms
358:	learn: 0.3126570	total: 2.45s	remaining: 962ms
359:	learn: 0.3125745	total: 2.46s	remaining: 955ms
360:	learn: 0.3125020	total: 2.46s	remaining: 948ms
361:	learn: 0.3124285	total: 2.47s	remaining: 941ms
362:	learn: 0.3123211	total: 2.47s	remaining: 934ms
363:	learn: 0.3122649	total: 2.48s	remaining: 927ms
364:	learn: 0.3121986	total: 2.49s	remaining: 920ms
365:	learn: 0.3121301	total: 2.49s	remaining: 913ms
366:	learn: 0.3120862	total: 2.5s	remaining: 906ms
367:	learn: 0.3120540	total: 2.5s	remaining: 899ms
368:	learn: 0.3119803	total: 2.52s	remaining: 895ms
369:	learn: 0.3119110	total: 2.53s	remaining: 888ms
370:	learn: 0.3118572	total: 2.53s	remaining: 881ms
371:	learn: 0.311

27:	learn: 0.3438956	total: 208ms	remaining: 3.5s
28:	learn: 0.3436363	total: 215ms	remaining: 3.49s
29:	learn: 0.3434315	total: 222ms	remaining: 3.47s
30:	learn: 0.3432220	total: 229ms	remaining: 3.46s
31:	learn: 0.3430312	total: 236ms	remaining: 3.45s
32:	learn: 0.3427922	total: 243ms	remaining: 3.44s
33:	learn: 0.3425766	total: 250ms	remaining: 3.43s
34:	learn: 0.3423765	total: 257ms	remaining: 3.41s
35:	learn: 0.3421566	total: 264ms	remaining: 3.4s
36:	learn: 0.3419756	total: 270ms	remaining: 3.38s
37:	learn: 0.3418163	total: 277ms	remaining: 3.37s
38:	learn: 0.3416450	total: 284ms	remaining: 3.35s
39:	learn: 0.3414215	total: 291ms	remaining: 3.34s
40:	learn: 0.3412039	total: 297ms	remaining: 3.33s
41:	learn: 0.3410064	total: 304ms	remaining: 3.32s
42:	learn: 0.3408065	total: 311ms	remaining: 3.31s
43:	learn: 0.3406246	total: 318ms	remaining: 3.29s
44:	learn: 0.3404641	total: 324ms	remaining: 3.28s
45:	learn: 0.3402585	total: 331ms	remaining: 3.27s
46:	learn: 0.3401219	total: 337ms

203:	learn: 0.3236665	total: 1.35s	remaining: 1.96s
204:	learn: 0.3236056	total: 1.36s	remaining: 1.95s
205:	learn: 0.3235348	total: 1.36s	remaining: 1.95s
206:	learn: 0.3234353	total: 1.37s	remaining: 1.94s
207:	learn: 0.3233349	total: 1.38s	remaining: 1.93s
208:	learn: 0.3232812	total: 1.38s	remaining: 1.93s
209:	learn: 0.3232445	total: 1.39s	remaining: 1.92s
210:	learn: 0.3231459	total: 1.4s	remaining: 1.91s
211:	learn: 0.3230771	total: 1.4s	remaining: 1.91s
212:	learn: 0.3230000	total: 1.41s	remaining: 1.9s
213:	learn: 0.3229170	total: 1.42s	remaining: 1.89s
214:	learn: 0.3228706	total: 1.42s	remaining: 1.89s
215:	learn: 0.3228153	total: 1.43s	remaining: 1.88s
216:	learn: 0.3227457	total: 1.44s	remaining: 1.87s
217:	learn: 0.3226766	total: 1.44s	remaining: 1.86s
218:	learn: 0.3226235	total: 1.45s	remaining: 1.86s
219:	learn: 0.3225477	total: 1.46s	remaining: 1.85s
220:	learn: 0.3224780	total: 1.46s	remaining: 1.85s
221:	learn: 0.3224335	total: 1.48s	remaining: 1.85s
222:	learn: 0.3

...

[iter 200] loss=0.2752 val_loss=0.0000 scale=1.0000 norm=0.5766
[iter 300] loss=0.2737 val_loss=0.0000 scale=1.0000 norm=0.5763
[iter 400] loss=0.2722 val_loss=0.0000 scale=1.0000 norm=0.5759
Fold :  0  Start
0:	learn: 2.6489607	total: 25.3ms	remaining: 481ms
1:	learn: 2.6263337	total: 45.7ms	remaining: 411ms
2:	learn: 2.6102705	total: 65.2ms	remaining: 369ms
3:	learn: 2.5958418	total: 84.4ms	remaining: 338ms
4:	learn: 2.5851114	total: 104ms	remaining: 312ms
5:	learn: 2.5723406	total: 123ms	remaining: 288ms
6:	learn: 2.5636358	total: 144ms	remaining: 268ms
7:	learn: 2.5519690	total: 164ms	remaining: 246ms
8:	learn: 2.5431732	total: 183ms	remaining: 223ms
9:	learn: 2.5361863	total: 202ms	remaining: 202ms
10:	learn: 2.5245923	total: 222ms	remaining: 181ms
11:	learn: 2.5208417	total: 241ms	remaining: 160ms
12:	learn: 2.5138942	total: 259ms	remaining: 140ms
13:	learn: 2.5043956	total: 279ms	remaining: 120ms
14:	learn: 2.4924205	total: 300ms	remaining: 100ms
15:	learn: 2.4827245	total: 319m

0:	learn: 2.6693713	total: 19.2ms	remaining: 366ms
1:	learn: 2.6449483	total: 38.1ms	remaining: 343ms
2:	learn: 2.6243512	total: 57.3ms	remaining: 325ms
3:	learn: 2.6084607	total: 76.5ms	remaining: 306ms
4:	learn: 2.5928091	total: 96ms	remaining: 288ms
5:	learn: 2.5808080	total: 116ms	remaining: 270ms
6:	learn: 2.5730967	total: 135ms	remaining: 251ms
7:	learn: 2.5621168	total: 155ms	remaining: 232ms
8:	learn: 2.5481822	total: 174ms	remaining: 212ms
9:	learn: 2.5369288	total: 193ms	remaining: 193ms
10:	learn: 2.5271477	total: 212ms	remaining: 174ms
11:	learn: 2.5173009	total: 232ms	remaining: 155ms
12:	learn: 2.5089393	total: 256ms	remaining: 138ms
13:	learn: 2.5009296	total: 278ms	remaining: 119ms
14:	learn: 2.4901334	total: 300ms	remaining: 99.9ms
15:	learn: 2.4819342	total: 322ms	remaining: 80.4ms
16:	learn: 2.4728459	total: 344ms	remaining: 60.7ms
17:	learn: 2.4635547	total: 364ms	remaining: 40.5ms
18:	learn: 2.4551760	total: 384ms	remaining: 20.2ms
19:	learn: 2.4486000	total: 404ms

16:	learn: 2.4064881	total: 255ms	remaining: 0us
Fold :  3  Start
0:	learn: 2.5259244	total: 33.9ms	remaining: 542ms
1:	learn: 2.5130526	total: 46.4ms	remaining: 348ms
2:	learn: 2.5001902	total: 59.1ms	remaining: 276ms
3:	learn: 2.4910241	total: 71.7ms	remaining: 233ms
4:	learn: 2.4833019	total: 83.8ms	remaining: 201ms
5:	learn: 2.4750060	total: 95.6ms	remaining: 175ms
6:	learn: 2.4664300	total: 108ms	remaining: 155ms
7:	learn: 2.4596719	total: 121ms	remaining: 136ms
8:	learn: 2.4532586	total: 134ms	remaining: 119ms
9:	learn: 2.4470452	total: 147ms	remaining: 103ms
10:	learn: 2.4413480	total: 160ms	remaining: 87.3ms
11:	learn: 2.4366545	total: 173ms	remaining: 72.2ms
12:	learn: 2.4310340	total: 187ms	remaining: 57.6ms
13:	learn: 2.4251172	total: 201ms	remaining: 43ms
14:	learn: 2.4197693	total: 214ms	remaining: 28.5ms
15:	learn: 2.4153432	total: 229ms	remaining: 14.3ms
16:	learn: 2.4110491	total: 241ms	remaining: 0us
Fold :  4  Start
0:	learn: 2.5276809	total: 68.5ms	remaining: 1.1s
1:

Fold :  5  Start
[iter 0] loss=2.2660 val_loss=0.0000 scale=1.0000 norm=2.0419
[iter 100] loss=2.2628 val_loss=0.0000 scale=1.0000 norm=2.0375
[iter 200] loss=2.2599 val_loss=0.0000 scale=1.0000 norm=2.0339
[iter 300] loss=2.2576 val_loss=0.0000 scale=1.0000 norm=2.0310
[iter 400] loss=2.2554 val_loss=0.0000 scale=1.0000 norm=2.0283
Fold :  6  Start
[iter 0] loss=2.2499 val_loss=0.0000 scale=1.0000 norm=2.0179
[iter 100] loss=2.2470 val_loss=0.0000 scale=1.0000 norm=2.0145
[iter 200] loss=2.2449 val_loss=0.0000 scale=1.0000 norm=2.0118
[iter 300] loss=2.2430 val_loss=0.0000 scale=1.0000 norm=2.0094
[iter 400] loss=2.2411 val_loss=0.0000 scale=1.0000 norm=2.0072
Fold :  7  Start
[iter 0] loss=2.2370 val_loss=0.0000 scale=1.0000 norm=1.9982
[iter 100] loss=2.2340 val_loss=0.0000 scale=1.0000 norm=1.9946
[iter 200] loss=2.2320 val_loss=0.0000 scale=1.0000 norm=1.9923
[iter 300] loss=2.2299 val_loss=0.0000 scale=1.0000 norm=1.9899
[iter 400] loss=2.2281 val_loss=0.0000 scale=1.0000 norm=1.

11:	learn: 1.6781578	total: 87.5ms	remaining: 7.29ms
12:	learn: 1.6535813	total: 93ms	remaining: 0us
Fold :  0  Start
Fold :  1  Start
Fold :  2  Start
Fold :  3  Start
Fold :  4  Start
Fold :  5  Start
Fold :  6  Start
Fold :  7  Start
Fold :  8  Start
Fold :  9  Start
Fold :  0  Start
Fold :  1  Start
Fold :  2  Start
Fold :  3  Start
Fold :  4  Start
Fold :  5  Start
Fold :  6  Start
Fold :  7  Start
Fold :  8  Start
Fold :  9  Start
Fold :  0  Start
[iter 0] loss=1.9736 val_loss=0.0000 scale=1.0000 norm=1.2179
[iter 100] loss=1.3012 val_loss=0.0000 scale=1.0000 norm=0.8868
[iter 200] loss=1.2443 val_loss=0.0000 scale=2.0000 norm=1.8104
[iter 300] loss=1.2289 val_loss=0.0000 scale=1.0000 norm=0.9160
[iter 400] loss=1.2220 val_loss=0.0000 scale=0.5000 norm=0.4601
Fold :  1  Start
[iter 0] loss=1.3605 val_loss=0.0000 scale=0.5000 norm=0.5333
[iter 100] loss=1.2266 val_loss=0.0000 scale=0.2500 norm=0.2338
[iter 200] loss=1.2228 val_loss=0.0000 scale=1.0000 norm=0.9310
[iter 300] loss=1

In [ ]:
sub = pd.read_csv('./sample_submission.csv')
sub['Y_01'] = stack_final1
sub['Y_02'] = stack_final2
sub['Y_03'] = stack_final3
sub['Y_04'] = stack_final4
sub['Y_05'] = stack_final5
sub['Y_06'] = stack_final6
sub['Y_07'] = stack_final7
sub['Y_08'] = stack_final8
sub['Y_09'] = stack_final9
sub['Y_10'] = stack_final10
sub['Y_11'] = stack_final11
sub['Y_12'] = stack_final12
sub['Y_13'] = stack_final13
sub['Y_14'] = stack_final14
sub.to_csv('./stack_.csv', index=False)

In [ ]:
## col1 col2 지정
stack_final.to_csv(f'{col1}_{col2}.csv', index=False)